In [66]:
#путь к данным
path = './data/server3.lpm.org.ru/~baulin/mmCIF/out/'

# Получаем список нужных файлов и цепей из NR-списка

In [67]:
#достаем данные из NR-лист

def get_files_chains(nr_list_file_name):
    filter_data = open(nr_list_file_name)
    filter_outfile = [] 
    
    for i, line in enumerate(filter_data):
        class_ = line.split('\",\"')[1].split('+')
        filter_outfile += class_

    filter_data.close()

    files_chains = {}
    for class_ in filter_outfile:
        class_ = class_.split('|')
        file_name = class_[0].lower() + '.out' + class_[1].strip()
        try:
            file = open(path + file_name)
        except IOError as e:
            continue
        else:
            with file:
                if file_name not in files_chains:
                    files_chains[file_name] = []
                files_chains[file_name].append(class_[2])
    return files_chains

# Строим набор компонент связности по файлу

In [68]:
import re
stem_re  = r'List of (\d+) stems'
stack_re = r'List of (\d+) stacks'
neighbor_re = r'Summary of structural features of (\d+) nucleotides'

def get_matrix(data, chains):

    #Достаем ребра из блока 'List of (\d+) stems'
    m_stem = re.search(stem_re, data)
    data_stems = ''
    stems = []
    if m_stem is not None:
        data_stems = data[m_stem.end():].split('****************************************************************************')[0]
        stems = data_stems.split('--------------------------------------------------------------------------')
    stems_output = []
    for stem in stems:
        stack1 = []
        stack2 = []
        for line in stem.split('\n'):
            line = line.split()
            if len(line) > 0:
                if line[0].isdigit():
                    stack1.append(line[1])
                    stack2.append(line[2])
        for i in range(len(stack1)-1):
            stems_output.append({'edge':(stack1[i], stack1[i+1]), 'type':'stem'} )
        for i in range(len(stack2)-1):
            stems_output.append({'edge':(stack2[i], stack2[i+1]), 'type':'stem'})


    #Достаем ребра из блока 'List of (\d+) stacks'
    m_stack = re.search(stack_re, data)
    data_stacks=''
    if m_stack is not None:
        data_stacks = data[m_stack.end():].split('****************************************************************************')[0]
    stacks_output = []
    for stack in data_stacks.split('\n'):
        stack = stack.strip().split()
        if len(stack)>0:
            if stack[0].isdigit():
                stack = stack[3].split(',')
                for i in range(len(stack)-1):
                    stacks_output.append({'edge':(stack[i], stack[i+1]), 'type':'stack'} )


    #Достаем ребра из блока 'Summary of structural features of (\d+) nucleotides'
    m_neighbor = re.search(neighbor_re, data) 
    data_neighbors=''
    if m_neighbor is not None:
        data_neighbors = data[m_neighbor.end():].split('****************************************************************************')[0]
    neighbor_output = []
    last_item = None
    for stack in data_neighbors.split('\n'):
        stack = stack.strip().split()
        if len(stack)>0:
            if stack[0].isdigit():
                if last_item is None:
                    last_item = stack[3]
                else:
                    if last_item.split('.')[:-3]==stack[3].split('.')[:-3]:
                        neighbor_output.append({'edge':( last_item, stack[3]), 'type':'neighbor'} )
                        last_item = stack[3]
                    else:
                        last_item = stack[3]


    #Все ребра между нуклеотидами (без связей внутри А-минора)
    edges = stacks_output+stems_output + neighbor_output


    #Достаем А-миноры 'List of (\d+) A-minor motifs'
    a_minor_re = r'List of (\d+) A-minor motifs'
    m_a_minor = re.search(a_minor_re, data) 
    a_minor_data=''
    
    if m_a_minor is not None:
        a_minor_data = data[m_a_minor.end():].split('****************************************************************************')[0]

            
            
#собираем граф        

    a_minors = {}
    a_minor_vert = set()
    last_item = None
    for stack in a_minor_data.split('\n'):
        stack = stack.strip().split()
        if len(stack)>0:
            if stack[0].isdigit():
                if stack[3].split('.')[2] in chains:
                    a_minor_vert.add(stack[3])
                f_stack, mid_stack = stack[3].split('|')
                s_stack, th_stack  = mid_stack.split(',')
                if f_stack not in a_minors:
                    a_minors[f_stack] = []
                a_minors[f_stack].append(stack[3])
                if s_stack not in a_minors:
                    a_minors[s_stack] = []
                a_minors[s_stack].append(stack[3])
                if th_stack not in a_minors:
                    a_minors[th_stack] = []
                a_minors[th_stack].append(stack[3])

    #Матрица смежности А-миноров

    matrix = {}
    matrix_type = {}
    for i, x in enumerate(a_minor_vert):
        matrix[x]={}
        matrix_type[x] = {}
        for y in a_minor_vert:
            if x<y:
                matrix[x][y]=[]
                matrix_type[x][y]=[]
                

    edges = stacks_output+stems_output + neighbor_output 


    for e in edges:
        a, b = min(e['edge']), max(e['edge'])
        if a in a_minors and b in a_minors:
            for x in a_minors[a]:
                if x in a_minor_vert:
                    for y in a_minors[b]:
                        if y in a_minor_vert:
                            if x < y:
                                matrix[x][y].append(e['type']+'_'+a+'_'+b)
                            if x > y:
                                matrix[y][x].append(e['type']+'_'+a+'_'+b)


    for x in matrix:
        matrix[x][x]=[]                                
                                
#связь между А-минорами по общим нуклеотидам
    for c in a_minors:
        if len(a_minors[c])>1:
            for i in range(len(a_minors[c])):
                if a_minors[c][i] in a_minor_vert:
                    for j in range(len(a_minors[c])):
                        if a_minors[c][j] in a_minor_vert:
                            if a_minors[c][i]<a_minors[c][j]:
                                matrix[a_minors[c][i]][a_minors[c][j]].append('a_minor_'+c)
                            else: 
                                matrix[a_minors[c][j]][a_minors[c][i]].append('a_minor_'+c)

    for x in matrix:
        for y in matrix:
            if x < y:
                matrix[y][x]=matrix[x][y]            
        matrix[x][x]=[]
    
    for x in matrix:
        for y in matrix:
            for t in matrix[x][y]:
                if t.find('a_minor_')!= -1:
                    matrix[y][x] = []
                    matrix[x][y] = []
    
    
    for x in matrix_type:
        for y in matrix_type:
            if x<y:
                eat = {}
                for e in matrix[x][y]:
                    et, e1, e2 = e.split('_')
                    ax, nnx = x.split('|')
                    lnx, rnx  = nnx.split(',')
                    ay, nny = y.split('|')
                    lny, rny  = nny.split(',')

                    type_str = ''
                    if lnx in (e1, e2):
                        type_str += 'L'
                    elif rnx in (e1, e2):
                        type_str += 'R'
                    elif ax in (e1, e2):
                        type_str += 'A'
                    if lny in (e1, e2):
                        type_str += 'L'
                    elif rny in (e1, e2):
                        type_str += 'R'
                    elif ay in (e1, e2):
                        type_str += 'A'

                    if type_str not in eat:
                        eat[type_str] = []
                    eat[type_str].append(et[0])
                    eat[type_str].sort()
                for e in eat:
                    new_t = ''.join(eat[e])+e
                    if x<y and new_t not in matrix_type[x][y]:
                        matrix_type[x][y].append(new_t)
                        matrix_type[x][y].sort()
                    if y<x:
                        matrix_type[y][x].append(new_t)
                        matrix_type[y][x].sort()
                        
                        
    #заполняем симметрично с учетом того, что изменилось "направление" ребер 
    for x in matrix_type:
        for y in matrix_type:
            if x < y:
                matrix_type[y][x]=[item[:-2]+item[-1]+ item[-2] for item in matrix_type[x][y]]
        matrix_type[x][x]=[]
        
        
    

    return matrix, matrix_type, len(a_minor_vert)


# Связность компонент

In [69]:
#наличие цикла в данной компоненте
has_cycle = 0
#порядок обхода
order = 0

def sub_dfs(u, matrix, visited, component):
    global has_cycle
    global order
    visited[u][0] = 1
    visited[u][2] = order
    order += 1
    numerator = {}
    
    for v in matrix[u]:
        if len(matrix[u][v])>0:
            a, b = min((u, v)), max((u, v))
            if {'edge': (a, b), 'links': matrix[a][b], 'edge_pattern':visited[a][1] + '_'+visited[b][1]} not in component:
                if visited[v][0] == 1:
                    has_cycle+=1
                component.append({'edge': (a, b), 'links': matrix[a][b], 'edge_pattern': visited[a][1] + '_'+visited[b][1]})
            if visited[v][0] == 0:
                sub_dfs(v, matrix, visited, component)
    visited[u][0] = 2
    

def dfs(matrix):
    global has_cycle
    global order
    text=[]
    cycle_text = []
    line_components = []
    cycle_components = []
    triv_components =[]
    visited = {ver:[0, 'v'+str(i), 0] for i, ver in enumerate(matrix)}
    component = []
    for v in matrix:            
        if not visited[v][0]:
            component = []
            has_cycle = 0
            sub_dfs(v, matrix, visited, component)
            if has_cycle > 0:
                cycle_components.append(component)
            elif len(component)>0:
                line_components.append(component)
            if len(component) == 0:
                triv_components.append(v)
            
    order = 0
    has_cycle = 0
    return  cycle_components, line_components, triv_components
    
    

# Вытаскиваем ветки

In [70]:
def get_line(comp):
    matrix = {}
    number_vert={}
    for i, e in enumerate(comp):
        v1, v2 = e['edge_pattern'].split('_')
        
        if v1 not in number_vert:
            number_vert[v1]=0
        number_vert[v1]+=1
        if v2 not in number_vert:
            number_vert[v2]=0
        number_vert[v2]+=1
        if v1 not in matrix:
            matrix[v1]={}
        if v2 not in matrix:
            matrix[v2]={}
        matrix[v1][v2] = [1]
        matrix[v2][v1] = [1]
    visited = {ver:[0, ver, 0] for i, ver in enumerate(matrix)}
    for v in number_vert:
        if number_vert[v]==1:
            if visited[v][0]==0:
                component = []
                sub_dfs(v, matrix, visited, component)
                for x in comp:
                    for i, y in enumerate(component):
                        x1, x2 = x['edge_pattern'].split('_')
                        y1, y2 = y['edge_pattern'].split('_')
                        #if visited[min(x1, x2)] > visited[max(x1, x2)]
                            
                
                        if min(x1, x2)==min(y1, y2) and max(x1, x2)==max(y1, y2):
                            #component[i]['edge']=
                            component[i]['edge']=x['edge']
                            component[i]['links']=x['links']
                            component[i]['type_pattern']=x['type_pattern']
                return component

# Генерация файлов

In [71]:
from datetime import datetime

import os

def make_files_with_components(result_path, nr_list_name):
    files_chains = get_files_chains(nr_list_name)

    directory = os.path.dirname(result_path)

    path_triv = result_path + 'triv' + os.sep
    path_line = result_path + 'line' + os.sep
    path_cycle = result_path + 'cycle' + os.sep
    all_cycle_components = []
    all_line_components = []
    all_triv_components = []

    try:
        os.stat(directory)
    except:
        os.mkdir(directory)   

    try:
        os.stat(os.path.dirname(path_triv))
    except:
        os.mkdir(os.path.dirname(path_triv))    

    try:
        os.stat(os.path.dirname(path_line))
    except:
        os.mkdir(os.path.dirname(path_line))   

    try:
        os.stat(os.path.dirname(path_cycle))
    except:
        os.mkdir(os.path.dirname(path_cycle)) 


    count_all = 0
    count_cycle = 0
    count_a_minor = 0
    count_line = 0
    f = open(result_path + 'unusual_line', 'w')
    type_patterns_in_simple_line=[]
    type_patterns_in_cycle=[]
    triv_components = []

    for i, x in enumerate(files_chains):

        text = []
        cycle_text = []
        line_text = []

        my_matrix = get_matrix(open(path + x).read(), files_chains[x])
        cycle_components, line_components, triv_components = dfs(my_matrix[0])                
        count_a_minor += my_matrix[2]
        for c in cycle_components:
            all_cycle_components.append(c)
            for i, e in enumerate(c):
                e['type_pattern'] = [t for t in set(my_matrix[1][e['edge'][0]][e['edge'][1]])]
                type_patterns_in_cycle.append(e['type_pattern'])
                c[i]=e
            cycle_text.append(str(c))
        for c in line_components:
            all_line_components.append(c)
            for i, e in enumerate(c):
                e['type_pattern'] = [t for t in set(my_matrix[1][e['edge'][0]][e['edge'][1]])]
                c[i]=e
                type_patterns_in_simple_line.append(e['type_pattern'] )
                for t in e['type_pattern']:
                    if t[-1]!=t[-2]:
                        f.write(str(e['type_pattern']) + '\t' + x.split('.')[0] + '_line\n')
            line_text.append(str(get_line(c)))
        if len(cycle_components)>0:
            count_cycle += len(cycle_components)
            f_comp = open(path_cycle + x.split('.')[0] + '_cycle', 'w')
            f_comp.write('\n'.join(cycle_text))
            f_comp.close()
        if len(line_components)>0:
            count_line += len(line_components)
            f_comp = open(path_line + x.split('.')[0] + '_line', 'w')
            f_comp.write('\n'.join(line_text))
            f_comp.close()
        if len(triv_components)>0:
            all_triv_components.append(c)
            f_comp = open(path_triv +x.split('.')[0] + '_triv', 'w')
            f_comp.write('\n'.join(triv_components))
            f_comp.close()
    f.close()
    return all_cycle_components, all_line_components, all_triv_components

# Получаем типы компонент

In [72]:
#c -- компонента, возвращаем (число а-миноров, число аденинов, число спариваний)
def get_type(c):
    ver = set()
    for e in c:
        ver.add(e['edge'][0])
        ver.add(e['edge'][1])
    ad = set()
    pr = set()
    for v in ver:
        a, p = v.split('|')
        ad.add(a)
        pr.add(p)
    return  len(ver), len(ad), len(pr)

def get_count(cs):
    ver = set()
    ad = set()
    pr = set()
    for c in cs:
        for e in c:
            ver.add(e['edge'][0])
            ver.add(e['edge'][1])

        for v in ver:
            a, p = v.split('|')
            ad.add(a)
            pr.add(p)
    return  len(ver), len(ad), len(pr)

# Обработка NR-файлов и генерация файлов с компонентами

In [73]:
import collections
def print_stat(comps, text):
    types= []
    print text
    for c in comps:
        types.append(get_type(c))
    counter_types = collections.Counter(types)
    print counter_types
    print 'Кол-во а-миноров, кол-во аденинов, кол-во спариваний, кол-во представителей типа'
    for i in counter_types.most_common(len(counter_types)):
        print i
        
    count = get_count(comps)
    print 'всего а-миноров: ', count[0]
    print 'всего аденинов: ', count[1]

In [74]:
import json

def collect_types(result_path, comps):
    
    path_line = result_path  + os.sep + 'line' + os.sep
    path_cycle = result_path  + os.sep + 'cycle' + os.sep
    path_all = result_path  + os.sep + 'all' + os.sep
    path_all_simple_format = result_path  + os.sep + 'all_simple_format' + os.sep

    try:
        os.stat(os.path.dirname(result_path))
    except:
        os.mkdir(os.path.dirname(result_path))   

    try:
        os.stat(os.path.dirname(path_line))
    except:
        os.mkdir(os.path.dirname(path_line))   

    try:
        os.stat(os.path.dirname(path_cycle))
    except:
        os.mkdir(os.path.dirname(path_cycle)) 
        
        
    try:
        os.stat(os.path.dirname(path_all))
    except:
        os.mkdir(os.path.dirname(path_all)) 
        
    try:
        os.stat(os.path.dirname(path_all_simple_format))
    except:
        os.mkdir(os.path.dirname(path_all_simple_format)) 
        
    types = {}
    
    for c in comps[1]:
        t = get_type(c)
        if t not in types:
            types[t] = []
        types[t].append(c)
    for t in types:
        with open(path_line+str(t), 'w') as fp:
            json.dump(types[t], fp)
    
    types = {}
    for c in comps[0]:
        t = get_type(c)
        if t not in types:
            types[t] = []
        types[t].append(c)
    for t in types:
        with open(path_cycle+str(t), 'w') as fp:
            json.dump(types[t], fp)
                   
         
    types = {}
    
    for c in comps[0] + comps[1]:
        t = get_type(c)
        if t not in types:
            types[t] = []
        types[t].append(c)
    for t in types:
        with open(path_all+str(t), 'w') as fp:
            json.dump(types[t], fp)
    for t in types:
        with open(path_all_simple_format+str(t), 'w') as fp:
            for c in types[t]:
                fp.write('—————————————————\npdbfile\nvertices\n')
                for x in c:
                    fp.write(x["edge_pattern"].split('_')[0] + '\t'+ x["edge"][0] + '\n')
                    fp.write(x["edge_pattern"].split('_')[1] + '\t'+ x["edge"][1] + '\n')
                fp.write('\nedges\n')
                for x in c:
                    fp.write(x["edge_pattern"].split('_')[0] + '\t'+x["edge_pattern"].split('_')[1] + '\t'+ str(x["type_pattern"]) + '\n')
                fp.write('\n')

In [75]:
components_nr_35a = make_files_with_components('./result_nr_list_nrlist_3.36_3.5A/', './data/nrlist_3.36_3.5A.csv')

In [76]:
print 'nrlist_3.36_3.5A:'

print '\tЦиклов: ', len(components_nr_35a[0]),\
      '\n\tВеток: ', len(components_nr_35a[1]),\
      '\n\tТривиальных компонент: ', len(components_nr_35a[2])

nrlist_3.36_3.5A:
	Циклов:  59 
	Веток:  830 
	Тривиальных компонент:  145


In [77]:
print_stat(components_nr_35a[0], 'nrlist_3.36_3.5A: Циклы')
print '\n'
print_stat(components_nr_35a[1], 'nrlist_3.36_3.5A: Ветки')

nrlist_3.36_3.5A: Циклы
Counter({(4, 4, 3): 14, (5, 4, 3): 13, (3, 3, 3): 3, (6, 4, 3): 3, (6, 6, 5): 3, (6, 5, 4): 3, (6, 6, 4): 3, (4, 4, 2): 3, (5, 5, 3): 2, (5, 5, 4): 2, (5, 4, 4): 2, (8, 7, 5): 1, (7, 6, 5): 1, (7, 6, 6): 1, (7, 7, 5): 1, (7, 5, 4): 1, (6, 5, 3): 1, (5, 4, 2): 1, (9, 9, 8): 1})
Кол-во а-миноров, кол-во аденинов, кол-во спариваний, кол-во представителей типа
((4, 4, 3), 14)
((5, 4, 3), 13)
((3, 3, 3), 3)
((6, 4, 3), 3)
((6, 6, 5), 3)
((6, 5, 4), 3)
((6, 6, 4), 3)
((4, 4, 2), 3)
((5, 5, 3), 2)
((5, 5, 4), 2)
((5, 4, 4), 2)
((8, 7, 5), 1)
((7, 6, 5), 1)
((7, 6, 6), 1)
((7, 7, 5), 1)
((7, 5, 4), 1)
((6, 5, 3), 1)
((5, 4, 2), 1)
((9, 9, 8), 1)
всего а-миноров:  300
всего аденинов:  260


nrlist_3.36_3.5A: Ветки
Counter({(2, 2, 2): 546, (3, 3, 3): 89, (3, 3, 2): 74, (4, 4, 4): 39, (4, 3, 2): 16, (5, 5, 5): 12, (6, 5, 6): 10, (4, 4, 3): 8, (5, 5, 4): 6, (4, 3, 3): 4, (7, 6, 6): 3, (3, 2, 3): 3, (5, 4, 4): 3, (4, 4, 2): 3, (8, 7, 7): 2, (6, 6, 6): 2, (7, 7, 7): 2, (8, 8,

In [78]:
collect_types('./data_35a', components_nr_35a) 

In [79]:
print_stat(components_nr_35a[0]+components_nr_35a[1], 'nrlist_3.36_3.5A: Суммарно')

nrlist_3.36_3.5A: Суммарно
Counter({(2, 2, 2): 546, (3, 3, 3): 92, (3, 3, 2): 74, (4, 4, 4): 39, (4, 4, 3): 22, (4, 3, 2): 16, (5, 4, 3): 13, (5, 5, 5): 12, (6, 5, 6): 10, (5, 5, 4): 8, (4, 4, 2): 6, (5, 4, 4): 5, (7, 6, 6): 4, (4, 3, 3): 4, (6, 5, 4): 4, (5, 5, 3): 3, (6, 4, 3): 3, (3, 2, 3): 3, (6, 6, 5): 3, (6, 6, 4): 3, (8, 7, 7): 2, (7, 7, 7): 2, (6, 6, 6): 2, (7, 6, 5): 1, (8, 7, 5): 1, (8, 8, 7): 1, (5, 4, 2): 1, (9, 9, 8): 1, (6, 5, 5): 1, (6, 5, 3): 1, (7, 5, 6): 1, (5, 4, 5): 1, (7, 7, 5): 1, (7, 5, 4): 1, (4, 2, 3): 1, (5, 3, 5): 1})
Кол-во а-миноров, кол-во аденинов, кол-во спариваний, кол-во представителей типа
((2, 2, 2), 546)
((3, 3, 3), 92)
((3, 3, 2), 74)
((4, 4, 4), 39)
((4, 4, 3), 22)
((4, 3, 2), 16)
((5, 4, 3), 13)
((5, 5, 5), 12)
((6, 5, 6), 10)
((5, 5, 4), 8)
((4, 4, 2), 6)
((5, 4, 4), 5)
((7, 6, 6), 4)
((4, 3, 3), 4)
((6, 5, 4), 4)
((5, 5, 3), 3)
((6, 4, 3), 3)
((3, 2, 3), 3)
((6, 6, 5), 3)
((6, 6, 4), 3)
((8, 7, 7), 2)
((7, 7, 7), 2)
((6, 6, 6), 2)
((7, 6, 5), 1